This notebook is converts filters in non votable form to votable format.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

This notebook was run with herschelhelp_internal version: 
284b2ef (Mon Aug 14 20:02:12 2017 +0100)


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import seaborn.apionly as sns

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux

from astropy.io.votable import parse
from astropy.io.votable import parse_single_table
from astropy.io import ascii
from astropy.io.votable.tree import VOTableFile, Resource, Field
from astropy.io.votable import from_table, writeto

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [4]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

filters_folder = "/Users/rs548/GitHub/herschelhelp_python/database_builder/filters/"

In [114]:
mosaic_ks = ascii.read(filters_folder + 'mosaic_ks.txt')

In [115]:
mosaic_ks

col1,col2
float64,float64
1.9,0.0
1.91,0.005
1.92,0.015
1.93,0.05
1.94,0.16
1.95,0.55
1.96,0.91
1.97,0.95
1.98,0.96


In [117]:
mosaic_ks['col1'].name = 'Wavelength'
# Convert units to angstroms
mosaic_ks['Wavelength'] = mosaic_ks['Wavelength']*1.e4
mosaic_ks['col2'].name = 'Transmission'

In [118]:
mosaic_ks

Wavelength,Transmission
float64,float64
19000.0,0.0
19100.0,0.005
19200.0,0.015
19300.0,0.05
19400.0,0.16
19500.0,0.55
19600.0,0.91
19700.0,0.95
19800.0,0.96


In [119]:
writeto(mosaic_ks, filters_folder + 'mosaic_ks.xml')

In [7]:
mosaic_b = ascii.read(filters_folder + 'mosaic_b.dat')
mosaic_b['col1'].name = 'Wavelength'
mosaic_b['col2'].name = 'Transmission'
mosaic_b

Wavelength,Transmission
float64,float64
3200.0,-0.01
3202.0,-0.015
3204.0,-0.005
3206.0,0.01
3208.0,-0.01
3210.0,-0.015
3212.0,-0.01
3214.0,-0.02
3216.0,-0.02


In [ ]:
writeto(mosaic_b, filters_folder + 'mosaic_b.xml')